## 多目标跟踪加权二分图匹配 ——KM(这里继续以找对象为例）

> 前馈知识：[多目标跟踪之二分图无权匹配——匈牙利算法](https://aistudio.baidu.com/aistudio/projectdetail/2037584)

### 1 算法流程

- 男女关系矩阵如下（正数表示权重，负数表示无关系）

|        |   女0  |     女1 |     女2 |     女3 |
| -------- | -------- | -------- | -------- |  -------- |
| 男0 | 8 | 6 | -1 | -1 |
| 男1 | -1 | 3 | 9 | -1 |
| 男2 | 9 | 8 | -1 | -1 |
| 男3 | -1 | -1 | 2 | -1 |
- 关系图如下，红线表示有关系（未匹配），蓝线表示已匹配，数值表示权重

![](https://ai-studio-static-online.cdn.bcebos.com/bfab51525e8c4cc79d83879a7a1fc22f7c37e0d6cd374755b88838c0a1280af7)


- 第一步：首先对每个顶点赋值，称为顶标，将左边的顶点赋值为与其相连的边的最大权重，右边的顶点赋值为0。

![](https://ai-studio-static-online.cdn.bcebos.com/79c451e5f02042f08be452f515328471dfdd8102f2fe46638e283b95148fca06)

- 第二步，开始匹配，匹配的原则是只和权重与左边分数（顶标）相同的边进行匹配，若找不到边匹配，对此条路径的所有左边顶点的顶标减d，所有右边顶点的顶标加d。参数d取1为例。

对于男0，与顶标分值相同的边先标蓝。

![](https://ai-studio-static-online.cdn.bcebos.com/edfd45aaa95f4952a3c1c0134278315b050369433a4c4e3da0b1f9bf240cf092)

然后是男1，与顶标分值相同的边标蓝。

![](https://ai-studio-static-online.cdn.bcebos.com/41927924278d409491fac06d6c63fc7e00716707f2ea4c79b45e7fb8d395cd2e)

- 第三步，然后是男2，发现与女0已经与男0配对。首先想到让男3更换匹配对象，然而根据匹配原则，只有权值大于等于9+0=9（左顶标加右顶标）的边能满足要求。于是男2无法换边。那男0能不能换边呢？对于男0来说，只有权值大于等于9+0=9的边能满足要求，无法换边。此时根据KM算法，应对所有冲突的边的顶点做加减操作，令左边顶点值减1，右边顶点值加1。结果如下图所示。

![](https://ai-studio-static-online.cdn.bcebos.com/5ce0c9d1dcd04275ad2bf923abfa5aa437aa1f7e27dd4a729fa126273b23b7a9)

- 第四步，再进行匹配操作，发现男2多了一条可匹配的边，因为此时男2对女1的匹配要求只需权重大于等于8+0即可，所以男2与女1匹配！

![](https://ai-studio-static-online.cdn.bcebos.com/bd9736b19b46400589f903bb45f64b9fa8a600fa3a964f34991bb21fddb651dd)

- 最后，最后进行男3的匹配，由于男3唯一的匹配对象女2已被男1匹配，发生冲突。进行一轮加减d操作，再匹配，男3还是匹配失败。两轮以后男3期望值降为0，放弃匹配男3。

### 2 代码实现

#### 2.1 定义KM方法类

In [1]:
import numpy as np

class KM_Algorithm_4:

    def __init__(self, Bipartite_Graph):

        self.Bipartite_Graph = Bipartite_Graph

        # 左右结点数量记录
        self.left = self.Bipartite_Graph.shape[0]  # 以左边为主
        # print(self.Bipartite_Graph)
        # print(self.Bipartite_Graph[0])
        self.right_true = self.Bipartite_Graph.shape[1]
        self.right = self.Bipartite_Graph.shape[1] + self.left

        self.reshape_graph()

        # 初始化顶标
        self.label_left = np.max(self.Bipartite_Graph, axis=1)  # 设置左边顶标为权重最大值（每行的最大值）

        self.label_right = np.zeros(self.right)  # 右边集合的顶标设置为0

        # 初始化辅助变量——是否已匹配
        self.visit_left = np.zeros(self.left, dtype=bool)
        self.visit_right = np.zeros(self.right, dtype=bool)

        # 初始化右边的匹配结果.如果已匹配就会对应匹配结果
        self.match_right = np.empty(self.right) * np.nan

        # 用inc记录需要减去的权值d，不断取最小值故初始化为较大值。权值都为负数，应该不用很大也行
        self.inc = 1000*1000*1000

        self.fail_boy = list()  # 每次匹配重新创建一个二分图匹配对象，所以这个也不用手动重置了

    def reshape_graph(self):
        new = np.ones((self.left, self.left)) * 0
        self.Bipartite_Graph = np.column_stack((self.Bipartite_Graph, new))
        # print(self.Bipartite_Graph)

    def match(self, boy):
        # print("---------------我是boy%d----------------------" % boy)
        boy = int(boy)
        # 记录下这个boy已经被寻找
        self.visit_left[boy] = True
        for girl in range(self.right):
            # 如果这个女生还没访问过
            if not self.visit_right[girl] and self.Bipartite_Graph[boy][girl] >= 0:  # 女孩仍未匹配并且男女之间存在匹配的可能性(不可匹配的点设置为负数，取反后变正数,故正数不可取)
                gap = self.label_left[boy] + self.label_right[girl] - self.Bipartite_Graph[boy][girl]  # gap也不会取到不能匹配的那条边
                if gap == 0:   # 差值为0，是可行的替换。所以可以直接尝试替换。后面不行再去将这个一起减去gap。这个列表是记录希望匹配的
                    self.visit_right[girl] = True
                    # 女生未被匹配，或虽然已被匹配，但是已匹配对象(男生)有其他可选备胎。这里那些是否已访问的列表不需要重置，因为不改变前面的尝试匹配
                    if np.isnan(self.match_right[girl]) or self.match(self.match_right[girl]):
                        self.match_right[girl] = boy
                        # print(self.match_right)
                        # 递归匹配，匹配成功
                        return 1

                # 找到权值最小的差距
                elif self.inc > gap:
                    self.inc = gap  # 等于0的gap不会存在这，所以只要存在可能匹配的情况，gap就不会等于原来的inc

        return 0

    def Kuh_Munkras(self):

        self.match_right = np.empty(self.right) * np.nan
        # 寻找最优匹配
        for man in range(self.left):
            while True:
                self.inc = 1000*1000  # the minimum gap
                self.reset()  # 每次寻找过的路径，所有要重置一下

                # 可找到可行匹配
                if self.match(man):
                    break
                # 不能找到可行匹配
                # (1)将所有在增广路中的boy方点的label全部减去最小常数
                # (2)将所有在增广路中的girl方点的label全部加上最小常数
                for k in range(self.left):
                    if self.visit_left[k]:
                        self.label_left[k] -= self.inc
                for n in range(self.right):
                    if self.visit_right[n]:
                        self.label_right[n] += self.inc

        return self.fail_boy

    def calculateSum(self):
        sum = 0
        boys_girls = []
        self.fail_boy = [i for i in range(self.left)]
        for i in range(self.right_true):
            if not np.isnan(self.match_right[i]):
                sum += self.Bipartite_Graph[int(self.match_right[i])][i]
                boy_girl = (int(self.match_right[i]), i)
                boys_girls.append(boy_girl)
                self.fail_boy.remove(int(self.match_right[i]))
        print("最短路径：", sum)

        return boys_girls, self.fail_boy

    def getResult(self):
        return self.match_right

    def reset(self):
        self.visit_left = np.zeros(self.left, dtype=bool)
        self.visit_right = np.zeros(self.right, dtype=bool)



#### 2.2 定义权重数值，执行主函数

In [3]:
# 定义权重数值
def data():
    graph = [[8,6,-1,-1],
            [-1,3,9,-1],
            [9,8,-1,-1],
            [-1,-1,2,-1]]
    #print(graph)
    km = KM_Algorithm_4(np.array(graph))

    km.Kuh_Munkras()  # 匹配

    boys_girls, fail_boys = km.calculateSum()  # 匹配结果元组,以及失败的男孩们

    print("匹配男女列表", boys_girls)
    print("失败男孩列表", fail_boys)


if __name__ == '__main__':
    data()

最短路径： 25.0
匹配男女列表 [(0, 0), (2, 1), (1, 2)]
失败男孩列表 [3]


### 小结
> - 以上是作者平时学习做的项目笔记，不同见解欢迎各位大佬指正
> - 奥利给
> - 如若存在问题，可在评论区留言，作者会不时为大家讲解
> - 作者aistudio主页链接，欢迎各位互粉、提问：[aistudio](https://aistudio.baidu.com/aistudio/personalcenter/thirdview/539945)